# Impyte Documentation
This is a first practicle attempt to clarify the usage of `impyte`. It's a collection of easy applicable and reproducible examples that you could use to simplyify your data processing workflow.

## Importing and generating sample data
In order to show some of the features of the library, we'll be using Kaggle's HR data that can be found [here](https://www.kaggle.com/ludobenistant/hr-analytics).

In [1]:
# import library and data set
import impyte
import pandas as pd
from tools.data_prep import remove_random

In [2]:
data = pd.read_csv('data/hr_test.csv')

In [3]:
data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


## Add random missing values
In order to show some of the pattern visualization and imputation methods, we need incomplete data sets. To achieve this, we'll be using a helper function that deletes values at random. 25 % of all values from each category will be deleted.

In [4]:
data = remove_random(data, .25, randomize_index=False)

In [5]:
data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,NaN,0.78,3.0,156.0,7.0,0.0,0.0,0.0,sales,medium
1,NaN,0.80,3.0,205.0,2.0,0.0,0.0,0.0,sales,low
2,NaN,0.57,2.0,160.0,3.0,0.0,1.0,0.0,support,low
3,NaN,0.67,3.0,150.0,2.0,0.0,0.0,1.0,sales,low
4,NaN,0.98,4.0,245.0,3.0,0.0,0.0,0.0,support,medium


In [6]:
data.describe()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years
count,14625.000000,14625.000000,14625.000000,14625.000000,14625.000000,14625.000000,14625.000000,14625.000000
mean,0.612738,0.716018,3.804103,201.091350,3.496684,0.144274,0.236855,0.021333
std,0.248770,0.171273,1.232474,49.945568,1.458209,0.351379,0.425167,0.144498
min,0.090000,0.360000,2.000000,96.000000,2.000000,0.000000,0.000000,0.000000
25%,0.440000,0.560000,3.000000,156.000000,3.000000,0.000000,0.000000,0.000000
50%,0.640000,0.720000,4.000000,200.000000,3.000000,0.000000,0.000000,0.000000
75%,0.820000,0.870000,5.000000,245.000000,4.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,7.000000,310.000000,10.000000,1.000000,1.000000,1.000000


In [7]:
len(data)

14999

In [8]:
reload(impyte)

<module 'impyte' from 'impyte.pyc'>

## Testing rudimentary features
Below is a first trial run of some of the features implemented by impyter and its helper classes.

## `NanChecker`
Functionality testing of `NanChecker` class.

In [9]:
nan_checker = impyte.NanChecker()

#### `NanChecker.is_nan(data, nan_vals=None, recursive=True)`
Detect missing values (NaN in numeric arrays, empty strings in string arrays). NaN detection feature also enables NaN-value ingestion as parameter values.

In [10]:
# [True, False, False]
print nan_checker.is_nan(["", 'None', 'NaN'])

# [True, True, True]
print nan_checker.is_nan(["", 'None', 'NaN'], nan_vals=['', None, 'None', 'NaN'])

[True, False, False]
[True, True, True]


In [11]:
# Recursive nan detection
# [True, True, False, [False, True, True]]
print nan_checker.is_nan(["", None, 'NaN', ["List Value 1", '', None]])

[True, True, False, [False, True, True]]


In [12]:
# Values can be declared as nan-values
# [True, False, False, True]
nan_checker.is_nan(['NaN', 'Empty', 'None', 'N/A'], nan_vals=['NaN', 'N/A'])

[True, False, False, True]

## `Pattern`
Functionality testing of `Pattern` class. The `Pattern` class stores different patterns and data summaries regarding NaN values. 

#### `Pattern._get_discrete_and_continuous(tmpdata)`
Returns the column names of discrete and continuous variables. Column names are stored in lists for easy selection. Those lists are stored in one dictionary object. All continuous column names can be accessed through `['continuous']` all discrete variables through `['discrete']`.

In [13]:
reload(impyte)
pattern_log = impyte.Pattern()

In [14]:
pattern_log._get_discrete_and_continuous(data)

{'continuous': ['satisfaction_level',
  'last_evaluation',
  'number_project',
  'average_montly_hours',
  'time_spend_company',
  'Work_accident',
  'left',
  'promotion_last_5years'],
 'discrete': ['sales', 'salary']}

#### `Pattern._compute_pattern(data, nan_values="", verbose=False)`
Function that checks for missing values and prints out a quick table of a summary of missing values. Includes pattern overview and counts of missing values by column. Computes pattern and returns its pattern structure plus the count of data points for each of these patterns. To get a nice digestible table add `["table"]` selector to output.

In [15]:
pattern_dict = pattern_log._compute_pattern(data)
indices, table = pattern_dict["indices"], pattern_dict["table"]

In [16]:
table

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,1,1,1,1,1,1,1,1,NaN,1,374
2,1,1,1,NaN,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
4,1,1,1,1,1,1,1,1,1,NaN,374
5,1,1,1,1,1,1,1,NaN,1,1,374
6,1,1,1,1,NaN,1,1,1,1,1,374
7,NaN,1,1,1,1,1,1,1,1,1,374
8,1,NaN,1,1,1,1,1,1,1,1,374
9,1,1,1,1,1,NaN,1,1,1,1,374


In [17]:
indices[0][:10] # first 10 indices of pattern 0

[3740, 3741, 3742, 3743, 3744, 3745, 3746, 3747, 3748, 3749]

In [18]:
pattern_nr = 0
print "Pattern {} has {:,} rows.".format(pattern_nr, len(indices[0])) 

Pattern 0 has 11,259 rows.


#### `Pattern.get_pattern()`
Returns NaN-patterns based on primary computation or initiates new computation of NaN-patterns. Uses `_compute_patterns()` to visualize patterns if not yet computed. Otherwise returns stored information regarding patterns.

In [19]:
pattern_log.get_pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,1,1,1,1,1,1,1,1,NaN,1,374
2,1,1,1,NaN,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
4,1,1,1,1,1,1,1,1,1,NaN,374
5,1,1,1,1,1,1,1,NaN,1,1,374
6,1,1,1,1,NaN,1,1,1,1,1,374
7,NaN,1,1,1,1,1,1,1,1,1,374
8,1,NaN,1,1,1,1,1,1,1,1,374
9,1,1,1,1,1,NaN,1,1,1,1,374


#### `Pattern.row_nan_pattern(row)`
This is a core piece to determining which patterns are available. Other methods work with the output of `row_nan_pattern()` to determine unique pattern structures and count these before turning them into readable tables.

In [20]:
# (1, 1, 'NaN', 'NaN')
print pattern_log.row_nan_pattern(['Value 1', 'Value 2', '', None])

(1, 1, 'NaN', 'NaN')


#### `Pattern.get_pattern_indices(pattern_no)`

In [21]:
pattern_log.get_pattern_indices(0)[:10] # get first 10 indices of pattern 0

[3740, 3741, 3742, 3743, 3744, 3745, 3746, 3747, 3748, 3749]

#### `Pattern.get_continuous()`
Returns list with names of all continuous variables.

In [22]:
pattern_log.get_continuous()

['satisfaction_level',
 'last_evaluation',
 'number_project',
 'average_montly_hours',
 'time_spend_company',
 'Work_accident',
 'left',
 'promotion_last_5years']

#### `Pattern.get_discrete()`
Returns list with names of all discrete variables.

In [23]:
pattern_log.get_discrete()

['sales', 'salary']

## `Impyter`

In [24]:
reload(impyte)
imputer = impyte.Impyter() # instantiate impyte class

### Load data into imputer

In [25]:
_ = imputer.load_data(data)

In [26]:
%timeit imputer.pattern()

The slowest run took 178768.43 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 5.01 µs per loop


In [27]:
%timeit imputer.get_pattern(4)

1000 loops, best of 3: 698 µs per loop


In [28]:
reload(impyte)
imputer = impyte.Impyter(data) # instantiate impyte class
nan_checker = impyte.NanChecker()
pattern_log = impyte.Pattern()
imputer.pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,1,1,1,1,1,1,1,1,NaN,1,374
2,1,1,1,NaN,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
4,1,1,1,1,1,1,1,1,1,NaN,374
5,1,1,1,1,1,1,1,NaN,1,1,374
6,1,1,1,1,NaN,1,1,1,1,1,374
7,NaN,1,1,1,1,1,1,1,1,1,374
8,1,NaN,1,1,1,1,1,1,1,1,374
9,1,1,1,1,1,NaN,1,1,1,1,374


In [29]:
_ = imputer.drop_pattern(4, inplace=True)

In [30]:
imputer.pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,1,1,1,1,1,1,1,1,NaN,1,374
2,1,1,1,NaN,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
5,1,1,1,1,1,1,1,NaN,1,1,374
6,1,1,1,1,NaN,1,1,1,1,1,374
7,NaN,1,1,1,1,1,1,1,1,1,374
8,1,NaN,1,1,1,1,1,1,1,1,374
9,1,1,1,1,1,NaN,1,1,1,1,374
10,1,1,1,1,1,1,NaN,1,1,1,374


In [31]:
_ = imputer.load_data(data)

In [32]:
imputer.pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,1,1,1,1,1,1,1,1,NaN,1,374
2,1,1,1,NaN,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
4,1,1,1,1,1,1,1,1,1,NaN,374
5,1,1,1,1,1,1,1,NaN,1,1,374
6,1,1,1,1,NaN,1,1,1,1,1,374
7,NaN,1,1,1,1,1,1,1,1,1,374
8,1,NaN,1,1,1,1,1,1,1,1,374
9,1,1,1,1,1,NaN,1,1,1,1,374


In [33]:
_ = imputer.drop_pattern(4)

In [34]:
imputer.pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,1,1,1,1,1,1,1,1,NaN,1,374
2,1,1,1,NaN,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
4,1,1,1,1,1,1,1,1,1,NaN,374
5,1,1,1,1,1,1,1,NaN,1,1,374
6,1,1,1,1,NaN,1,1,1,1,1,374
7,NaN,1,1,1,1,1,1,1,1,1,374
8,1,NaN,1,1,1,1,1,1,1,1,374
9,1,1,1,1,1,NaN,1,1,1,1,374


In [35]:
imputer.pattern_log.easy_access

{(1, 1, 1, 1, 1, 1, 1, 1, 1, 'NaN'): ['salary'],
 (1, 1, 1, 1, 1, 1, 1, 1, 'NaN', 1): ['sales'],
 (1, 1, 1, 1, 1, 1, 1, 'NaN', 1, 1): ['promotion_last_5years'],
 (1, 1, 1, 1, 1, 1, 'NaN', 1, 1, 1): ['left'],
 (1, 1, 1, 1, 1, 'NaN', 1, 1, 1, 1): ['Work_accident'],
 (1, 1, 1, 1, 'NaN', 1, 1, 1, 1, 1): ['time_spend_company'],
 (1, 1, 1, 'NaN', 1, 1, 1, 1, 1, 1): ['average_montly_hours'],
 (1, 1, 'NaN', 1, 1, 1, 1, 1, 1, 1): ['number_project'],
 (1, 'NaN', 1, 1, 1, 1, 1, 1, 1, 1): ['last_evaluation'],
 ('NaN', 1, 1, 1, 1, 1, 1, 1, 1, 1): ['satisfaction_level']}

### Comparison of complete indices methods

In [36]:
%timeit len(imputer.pattern_log.get_complete_indices())

1000 loops, best of 3: 958 µs per loop


In [37]:
%timeit len(imputer.get_complete_old())

100 loops, best of 3: 6.47 ms per loop


In [38]:
_ = imputer.drop_pattern(7, inplace=True)

In [39]:
data.iloc[:, imputer.pattern_log.store_tuple_columns[(1, 1, 1, 1, 1, 1, 1, 1, 1, 'NaN')]].columns

Index([u'salary'], dtype='object')

In [40]:
data.tail()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
14994,0.66,0.92,4.0,239.0,3.0,0.0,0.0,0.0,hr,medium
14995,0.09,0.98,6.0,271.0,4.0,0.0,1.0,0.0,sales,low
14996,0.80,0.85,4.0,139.0,2.0,0.0,0.0,0.0,product_mng,medium
14997,0.87,0.67,4.0,272.0,4.0,0.0,0.0,0.0,sales,low
14998,1.00,0.49,3.0,185.0,2.0,0.0,0.0,0.0,sales,low


### Timing some of the functions
Below is a piece of information on efficiency of the functions. This doesn't reflect complexity of the functions, but it will show an runtime estimate for your local machine.

In [41]:
%timeit nan_checker.is_nan(["", 'None', 'NaN'])
%timeit nan_checker.is_nan(["", 'None', 'NaN'], nan_vals=['', None, 'None', 'NaN'])

The slowest run took 15.76 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 5.21 µs per loop
The slowest run took 4.26 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 5.21 µs per loop


### `impute()`
Impute is the core method of impyte. The method works out of the box and uses Random Forest estimators per default to impute missing values. It automatically performs cross-validation to showcase the potential accuracy of the imputation.

In [42]:
reload(impyte)
imputer = impyte.Impyter()
imputer.load_data(data)
_ = imputer.pattern()
imputer.impute()

Label: sales 	 Fitting RandomForestClassifier
CV-Scores: [ 0.36569149  0.35879218  0.37272323  0.37866667  0.37688889]
['management' 'support']
Label: average_montly_hours 	 Fitting RandomForestRegressor
CV-Scores: [ 0.29150737  0.2741849   0.29629812  0.28465866  0.29202629]
[ 187.8  208.6]
Label: number_project 	 Fitting RandomForestRegressor
CV-Scores: [ 0.48801177  0.46570469  0.49844656  0.48101647  0.46984268]
[ 3.   3.9]
Label: salary 	 Fitting RandomForestClassifier
CV-Scores: [ 0.55836662  0.57548845  0.56927176  0.60328743  0.58107508]
['low' 'medium']
Label: promotion_last_5years 	 Fitting RandomForestRegressor
CV-Scores: [ 0.2487236   0.19711804  0.20556124  0.15317759  0.23354286]
[ 0.  0.]
Label: time_spend_company 	 Fitting RandomForestRegressor
CV-Scores: [ 0.32306808  0.36112498  0.30046586  0.30011884  0.33845951]
[ 3.   3.4]
Label: satisfaction_level 	 Fitting RandomForestRegressor
CV-Scores: [ 0.45473078  0.41959276  0.42690524  0.41835146  0.42048714]
[ 0.577  0.83

(['satisfaction_level',
  'last_evaluation',
  'number_project',
  'average_montly_hours',
  'time_spend_company',
  'Work_accident',
  'left',
  'promotion_last_5years'],
 ['sales', 'salary'])

In [43]:
data['last_evaluation'].describe()

count    14625.000000
mean         0.716018
std          0.171273
min          0.360000
25%          0.560000
50%          0.720000
75%          0.870000
max          1.000000
Name: last_evaluation, dtype: float64

In [44]:
from sklearn.preprocessing import StandardScaler

In [45]:
scaler = StandardScaler()

In [47]:
test = data['last_evaluation'].dropna()

In [56]:
test[test.apply(nan_checker.is_nan)]

Series([], Name: last_evaluation, dtype: float64)

In [57]:
test.describe()

count    14625.000000
mean         0.716018
std          0.171273
min          0.360000
25%          0.560000
50%          0.720000
75%          0.870000
max          1.000000
Name: last_evaluation, dtype: float64

In [61]:
test_scaled = scaler.fit_transform(test.values.reshape(-1, 1))

In [66]:
test[:5]

0    0.78
1    0.80
2    0.57
3    0.67
4    0.98
Name: last_evaluation, dtype: float64

In [65]:
test_scaled[:5]

array([[ 0.37358184],
       [ 0.49035861],
       [-0.85257418],
       [-0.26869036],
       [ 1.54134948]])

In [67]:
test_rescaled = scaler.inverse_transform(test_scaled)

In [68]:
test_rescaled

array([[ 0.78],
       [ 0.8 ],
       [ 0.57],
       ..., 
       [ 0.85],
       [ 0.67],
       [ 0.49]])